添加包 
抑制报错

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

np.set_printoptions(suppress=True)
import warnings 
warnings.filterwarnings('ignore')


In [ ]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

数据分析

In [ ]:
train.head()

In [ ]:
train.info()
train.Stay.unique()

In [ ]:
train.isnull().sum().sort_values(ascending=False)


In [ ]:
test.isnull().sum().sort_values(ascending=False)

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
for i in train.columns:
    print(i, ':', train[i].nunique())
    
# nunique 返回不同值的个数
# unique 返回不同值的数组

In [ ]:
# Number of distinct observations in test dataset
for i in test.columns:
    print(i, ':', test[i].nunique())

data preparation 

In [ ]:
train['Bed Grade'].fillna(train['Bed Grade'].mode()[0],inplace=True)
test['Bed Grade'].fillna(test['Bed Grade'].mode()[0],inplace=True)

train['City_Code_Patient'].fillna(train['City_Code_Patient'].mode()[0], inplace = True)
test['City_Code_Patient'].fillna(test['City_Code_Patient'].mode()[0], inplace = True)
# mode 众数 
# param(true) 默认不考虑缺省值

label encoding Stay column in train dataset 

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['Stay'] = le.fit_transform(train['Stay'].astype('str'))

In [ ]:
train.head()

In [ ]:
test.columns


In [ ]:
#Imputing dummy Stay column in test datset to concatenate with train dataset
test['Stay'] = -1 
df = pd.concat([train, test])
df.shape

# 这里的test以全部都是-1填充

In [ ]:
#Label Encoding all the columns in Train and test datasets
for i in ['Hospital_type_code', 'Hospital_region_code', 'Department',
          'Ward_Type', 'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness', 'Age']:
    le = LabelEncoder()
    df[i] = le.fit_transform(df[i].astype(str))

In [ ]:
#Spearating Train and Test Datasets
train = df[df['Stay']!=-1]
test = df[df['Stay']==-1]

feature engineering 

In [ ]:
def get_countid_enocde(train, test, cols, name):
  temp = train.groupby(cols)['case_id'].count().reset_index().rename(columns = {'case_id': name})
  train = pd.merge(train, temp, how='left', on= cols)
  train[name] = train[name].astype('float')
  train[name].fillna(np.median(temp[name]), inplace = True)
  
  temp2 = test.groupby(cols)['case_id'].count().reset_index().rename(columns = {'case_id': name})
  test = pd.merge(test,temp2, how='left', on= cols)
  test[name] = test[name].astype('float')
  test[name].fillna(np.median(temp2[name]), inplace = True)
  return train, test

In [ ]:
train, test = get_countid_enocde(train, test, ['patientid'], name = 'count_id_patient')
train, test = get_countid_enocde(train, test, 
                                 ['patientid', 'Hospital_region_code'], name = 'count_id_patient_hospitalCode')
train, test = get_countid_enocde(train, test, 
                                 ['patientid', 'Ward_Facility_Code'], name = 'count_id_patient_wardfacilityCode')

In [ ]:
# Droping duplicate columns
test1 = test.drop(['Stay', 'patientid', 'Hospital_region_code', 'Ward_Facility_Code'], axis =1)
train1 = train.drop(['case_id', 'patientid', 'Hospital_region_code', 'Ward_Facility_Code'], axis =1)

In [ ]:
# Splitting train data for Naive Bayes and XGBoost
X1 = train1.drop('Stay', axis =1)
y1 = train1['Stay']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size =0.20, random_state =100)

构建模型

In [ ]:
from sklearn.naive_bayes import GaussianNB
target = y_train.values
features = X_train.values
classifier_nb = GaussianNB()
model_nb = classifier_nb.fit(features, target)


In [ ]:
prediction_nb = model_nb.predict(X_test)
from sklearn.metrics import accuracy_score
acc_score_nb = accuracy_score(prediction_nb,y_test)
print("Acurracy:", acc_score_nb*100)

In [ ]:
import xgboost
classifier_xgb = xgboost.XGBClassifier(max_depth=4, learning_rate=0.1, n_estimators=800,
                                  objective='multi:softmax', reg_alpha=0.5, reg_lambda=1.5,
                                  booster='gbtree', n_jobs=4, min_child_weight=2, base_score= 0.75)

In [ ]:
model_xgb = classifier_xgb.fit(X_train, y_train)

In [ ]:
prediction_xgb = model_xgb.predict(X_test)
acc_score_xgb = accuracy_score(prediction_xgb,y_test)
print("Accuracy:", acc_score_xgb*100)

In [ ]:
# Segregation of features and target variable
X = train.drop('Stay', axis =1)
y = train['Stay']
print(X.columns)
z = test.drop('Stay', axis = 1)
print(z.columns)

# Data Scaling
from sklearn import preprocessing
X_scale = preprocessing.scale(X)
X_scale.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size =0.20, random_state =100)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [ ]:
from keras.utils import to_categorical
#Sparse Matrix
a = to_categorical(y_train)
b = to_categorical(y_test)

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape = (254750, 20))) 
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(11, activation='softmax'))

model.summary()

model.compile(optimizer= 'SGD', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

callbacks = [tf.keras.callbacks.TensorBoard("logs_keras")]
model.fit(X_train, a, epochs=20, callbacks=callbacks, validation_split = 0.2)

In [ ]:
model = Sequential()
model.add(tf.keras.Input(shape=(20,)))
model.add(Dense(64, activation='relu')) 
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(11, activation='softmax'))

model.compile(optimizer= 'SGD', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
callbacks = [tf.keras.callbacks.TensorBoard("logs_keras")]
model.fit(X_train, a, epochs=20, callbacks=callbacks, validation_split = 0.2)

In [ ]:
# Retraining the model with 4 epochs
model.fit(X_train, a, epochs=4, validation_split = 0.2)
print("\n Model Evaluation")
model.evaluate(X_test,b)